In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import os , json 


C:\Users\oscar\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [3]:
data = pd.read_json("../data/raw_dataset/Entity-Recoginition-in-Resume.json", lines = True) 
data.head()

,content,annotation,extras
0,Abhishek Jha\nApplication Development Associat...,"[{'label': ['Skills'], 'points': [{'start': 12...",NaN
1,Afreen Jamadar\nActive member of IIIT Committe...,"[{'label': ['Email Address'], 'points': [{'sta...",NaN
2,"Akhil Yadav Polemaina\nHyderabad, Telangana - ...","[{'label': ['Skills'], 'points': [{'start': 37...",NaN
3,Alok Khandai\nOperational Analyst (SQL DBA) En...,"[{'label': ['Skills'], 'points': [{'start': 80...",NaN
4,Ananya Chavan\nlecturer - oracle tutorials\n\n...,"[{'label': ['Degree'], 'points': [{'start': 20...",NaN


In [4]:
# cheack an example
pd.options.display.max_columns = None

In [5]:
print(data.iloc[0, 0])

Abhishek Jha
Application Development Associate - Accenture

Bengaluru, Karnataka - Email me on Indeed: indeed.com/r/Abhishek-Jha/10e7a8cb732bc43a

• To work for an organization which provides me the opportunity to improve my skills
and knowledge for my individual and company's growth in best possible ways.

Willing to relocate to: Bangalore, Karnataka

WORK EXPERIENCE

Application Development Associate

Accenture -

November 2017 to Present

Role: Currently working on Chat-bot. Developing Backend Oracle PeopleSoft Queries
for the Bot which will be triggered based on given input. Also, Training the bot for different possible
utterances (Both positive and negative), which will be given as
input by the user.

EDUCATION

B.E in Information science and engineering

B.v.b college of engineering and technology -  Hubli, Karnataka

August 2013 to June 2017

12th in Mathematics

Woodbine modern school

April 2011 to March 2013

10th

Kendriya Vidyalaya

April 2001 to March 2011

SKILLS

C (Less

In [6]:
from pprint import pprint
pprint(data.iloc[0, 1][0]['points'][0]['start'])

1295


In [38]:
def get_start_end_label(entittes):
    results = []
    for item in entittes:
        try:
            label = item['label'][0] 
        except :
            label = item['label']
            
        start = item['points'][0]['start']
        end = item['points'][0]['end']
        results.append((start , end, label))
    return sorted(results , key=lambda x :x[0], reverse=False)

        

In [39]:
ents = get_start_end_label(data.iloc[0, 1])
ents

[(0, 11, 'Name'),
 (13, 45, 'Designation'),
 (49, 57, 'Companies worked at'),
 (60, 68, 'Location'),
 (95, 145, 'Email Address'),
 (372, 404, 'Designation'),
 (407, 415, 'Companies worked at'),
 (727, 769, 'Designation'),
 (771, 813, 'College Name'),
 (856, 860, 'Graduation Year'),
 (883, 904, 'College Name'),
 (939, 956, 'College Name'),
 (993, 1153, 'Skills'),
 (1295, 1621, 'Skills')]

In [40]:
data.iloc[0, 0][12:12]

''

In [41]:
def fill_gap(sorted_entities , len_text):
    sorted_entities_copy = sorted_entities.copy()
    i = 0
    prev_start , prev_end , _ = sorted_entities_copy.pop(0)
    
    if prev_start !=0:
        sorted_entities.insert(0 , (0 , prev_start-1 , 'O'))
        i+=1
    for start , end  , _ in sorted_entities_copy:
        diff = end - prev_start
        if diff == 1:
            prev_start = start 
            prev_end = end
            i+=1
            continue
        else :
            sorted_entities.insert(i , (prev_end + 1 , start -1 , 'O'))
            prev_end = end
            prev_start = start
            i+=1
    if end != len_text:
        sorted_entities.insert(-1 , (end+1 , len_text , 'O'))
    return sorted(sorted_entities , key=lambda x:x[1])

            
        

In [42]:
fill_gap(ents , 2000)

[(0, 11, 'Name'),
 (12, 12, 'O'),
 (13, 45, 'Designation'),
 (46, 48, 'O'),
 (49, 57, 'Companies worked at'),
 (58, 59, 'O'),
 (60, 68, 'Location'),
 (69, 94, 'O'),
 (95, 145, 'Email Address'),
 (146, 371, 'O'),
 (372, 404, 'Designation'),
 (405, 406, 'O'),
 (407, 415, 'Companies worked at'),
 (416, 726, 'O'),
 (727, 769, 'Designation'),
 (770, 770, 'O'),
 (771, 813, 'College Name'),
 (814, 855, 'O'),
 (856, 860, 'Graduation Year'),
 (861, 882, 'O'),
 (883, 904, 'College Name'),
 (905, 938, 'O'),
 (939, 956, 'College Name'),
 (957, 992, 'O'),
 (993, 1153, 'Skills'),
 (1154, 1294, 'O'),
 (1295, 1621, 'Skills'),
 (1622, 2000, 'O')]

In [43]:
def flatten_entities(text , entittes):
    
    len_text = len(text)
    len_all_words = len(text.split())
    sorted_entities = get_start_end_label(entittes)
    sorted_with_gap = fill_gap(sorted_entities , len_text)
    all_labels = []
    
    for start , end , label in sorted_with_gap:
        if start == end:
            continue
        sub_words = text[start : end]
        len_sub_words = len(sub_words.split())
        
        if label == 'O':
            
            all_labels.extend(['O']*len_sub_words)
        else:
            if len_sub_words > 1:
                all_labels.extend([f'B-{label}']+[f'I-{label}']*(len_sub_words - 1))
            else:
                all_labels.append(f'B-{label}')
    while len(all_labels) > len_all_words:
        all_labels.pop(-1)
    while len(all_labels) < len_all_words:
        all_labels.append('O')
    return all_labels

In [44]:
res = flatten_entities(data.iloc[0,0], data.iloc[0,1])

In [45]:
data['ner-tags'] = [flatten_entities(text , entittes) for text , entittes in\
                    zip(data['content'] , data['annotation'])]

In [46]:
assert all(data['ner-tags'].apply(len) ==data['content'].str.split().apply(len))

In [107]:
data['ner-tags']

0      [B-Name, I-Name, B-Designation, I-Designation,...
1      [B-Name, I-Name, O, O, O, O, O, O, O, O, B-Loc...
2      [B-Name, I-Name, I-Name, B-Location, O, O, O, ...
3      [B-Name, I-Name, B-Designation, I-Designation,...
4      [B-Name, I-Name, B-Designation, O, I-Companies...
                             ...                        
215    [B-Name, I-Name, B-Designation, B-Location, O,...
216    [B-Name, I-Name, B-Designation, I-Designation,...
217    [B-Name, I-Name, B-Designation, I-Designation,...
218    [B-Name, I-Name, B-Designation, I-Designation,...
219    [B-Name, I-Name, B-Designation, I-Designation,...
Name: ner-tags, Length: 220, dtype: object

In [108]:
all_tags =set()
for tags in data['ner-tags']:
    all_tags = all_tags.union(set(tags))
print(all_tags)

{'B-Years of Experience', 'I-College Name', 'I-Skills', 'I-Designation', 'I-Name', 'B-Graduation Year', 'B-Companies worked at', 'I-UNKNOWN', 'B-College Name', 'B-Email Address', 'I-Companies worked at', 'B-Location', 'B-Skills', 'B-Name', 'B-Degree', 'I-Graduation Year', 'I-Degree', 'I-Location', 'I-Email Address', 'O', 'B-Designation', 'I-Years of Experience'}


In [110]:
for i , j in zip(data.iloc[5 , 0].split() ,data.iloc[5, 3] ):
    print(i , '---->' , j)

Anvitha ----> B-Name
Rao ----> I-Name
Automation ----> B-Designation
developer ----> I-Designation
- ----> O
Email ----> O
me ----> O
on ----> I-Designation
Indeed: ----> O
indeed.com/r/Anvitha-Rao/9d6acc68cc30c71c ----> B-Email Address
Seeking ----> O
a ----> I-Name
software ----> O
development ----> O
internship ----> O
position ----> O
for ----> O
Summer ----> O
2018 ----> O
that ----> O
utilizes ----> O
my ----> O
technical ----> O
skills, ----> O
education ----> O
and ----> O
passion ----> O
for ----> O
solving ----> O
interesting ----> O
problems ----> O
as ----> O
a ----> I-Name
software ----> O
professional. ----> O
Willing ----> O
to ----> I-Designation
relocate: ----> O
Anywhere ----> O
WORK ----> O
EXPERIENCE ----> O
Automation ----> B-Designation
developer ----> I-Designation
SAP ----> B-Companies worked at
Labs ----> I-Companies worked at
- ----> O
Bengaluru, ----> B-Location
Karnataka ----> O
- ----> O
August ----> O
2016 ----> B-Graduation Year
to ----> I-Designation
Aug

In [112]:
data.iloc[5, 1]

[{'label': ['Skills'],
  'points': [{'start': 2826,
    'end': 2878,
    'text': 'IoT, MySQL, PostgreSQL, D3js, Hadoop and Spark, Gephi'}]},
 {'label': ['Skills'],
  'points': [{'start': 2768,
    'end': 2809,
    'text': 'C, C++, HTML/CSS, Java, Python, Javascript'}]},
 {'label': ['Graduation Year'],
  'points': [{'start': 2513, 'end': 2516, 'text': '2016'}]},
 {'label': ['Location'],
  'points': [{'start': 2468, 'end': 2476, 'text': 'Bengaluru'}]},
 {'label': ['College Name'],
  'points': [{'start': 2429,
    'end': 2463,
    'text': 'M S Ramaiah Institute of Technology'}]},
 {'label': ['Degree'],
  'points': [{'start': 2384,
    'end': 2427,
    'text': 'Bachelor of Engineering in Computer Science\n'}]},
 {'label': ['Graduation Year'],
  'points': [{'start': 2378, 'end': 2381, 'text': '2019'}]},
 {'label': ['College Name'],
  'points': [{'start': 2329,
    'end': 2352,
    'text': 'Arizona State University'}]},
 {'label': ['Degree'],
  'points': [{'start': 2300,
    'end': 2327,
   

In [48]:
mapping = {'O': 'O',
 'B-Years of Experience': 'B-YOE',
 'I-College Name': 'I-COL',
 'I-Name': 'I-Name',
 'I-Skills': 'I-Skills',
 'I-Designation': 'I-Des',
 'B-Graduation Year': 'B-GYear',
 'B-Email Address': 'B-Email',
 'I-Companies worked at': 'I-CWA',
 'B-College Name': 'B-COL',
 'I-Email Address': 'I-Email',
 'B-Companies worked at': 'B-CWA',
 'B-Location': 'B-LOC',
 'B-Skills': 'B-Skills',
 'B-Name': 'B-Name',
 'B-Degree': 'B-Degree',
 'I-Location': 'I-LOC',
 'I-Degree': 'I-Degree',
 'I-Graduation Year': 'I-GYear',
 'B-Designation': 'B-Des',
 'I-Years of Experience': 'I-YOE'}

In [49]:
import pickle
with open("../mapping.pkl", 'wb') as f:
    pickle.dump(mapping , f)
  